# TNFR Atoms and Molecules Study — Nodal dynamics to coherent patterns

This notebook summarizes how the nodal equation (∂EPI/∂t = νf · ΔNFR) and operator grammar explain the existence of element-like patterns (H, C, N, O) and a metal-like pattern (Au) as coherent attractors.

We compute telemetry-only Structural Field Tetrad summaries for Z ∈ {1,6,7,8,79} and compare. Optionally, we compose an Au-like network to observe longer ξ_C. All exports are reproducible.

Note: Language standard is English. For canonical computational mathematics and cross-links to experiments, see `src/tnfr/mathematics/README.md`. For formal derivations, see `docs/source/theory/mathematical_foundations.md`.

## Physics sketch (TNFR)
- Existence = coherence: bounded integral ∫ νf·ΔNFR dt and operator sequences obeying U1–U5.
- U3 (resonant coupling): explicit phase verification ensures links form only with |Δφ| ≤ Δφ_max.
- Telemetry (U6): Φ_s confinement (ΔΦ_s), local stress |∇φ|, curvature K_φ, coherence length ξ_C.

In [ ]:
from __future__ import annotations

import os, json
import pandas as pd

from tnfr.examples_utils import build_element_radial_graph, apply_synthetic_activation_sequence
from tnfr.physics.fields import compute_structural_potential, estimate_coherence_length
from tnfr.operators.grammar import (
    warn_phase_gradient_telemetry, warn_phase_curvature_telemetry, warn_coherence_length_telemetry,
    validate_structural_potential_confinement,
 )
from tnfr.telemetry.constants import PHASE_GRADIENT_THRESHOLD, PHASE_CURVATURE_ABS_THRESHOLD, STRUCTURAL_POTENTIAL_DELTA_THRESHOLD

OUTPUT_DIR = os.path.join('examples','output'); os.makedirs(OUTPUT_DIR, exist_ok=True)
Z_LIST = [1,6,7,8,79]
SYMBOLS = {1:'H',6:'C',7:'N',8:'O',79:'Au'}

def summarize_graph(G, label):
    phi_before = compute_structural_potential(G)
    _, sg, _, _ = warn_phase_gradient_telemetry(G, threshold=PHASE_GRADIENT_THRESHOLD)
    _, sk, _, _ = warn_phase_curvature_telemetry(G, abs_threshold=PHASE_CURVATURE_ABS_THRESHOLD, multiscale_check=True, alpha_hint=2.76)
    _, sx, _ = warn_coherence_length_telemetry(G)
    xi_c = float(estimate_coherence_length(G))
    apply_synthetic_activation_sequence(G, alpha=0.25, dnfr_factor=0.9)
    phi_after = compute_structural_potential(G)
    ok, drift, _ = validate_structural_potential_confinement(G, phi_before, phi_after, threshold=STRUCTURAL_POTENTIAL_DELTA_THRESHOLD, strict=False)
    return dict(label=label, xi_c=xi_c, mean_grad=float(sg.get('mean_abs',0.0)), mean_kphi=float(sk.get('mean_abs',0.0)), mean_path_length=float(sx.get('mean_path_length',0.0)), u6_ok=bool(ok), u6_drift=float(drift))

rows = []
for Z in Z_LIST:
    G = build_element_radial_graph(Z, seed=100+Z)
    label = SYMBOLS.get(Z, str(Z))
    rows.append({'Z': Z, 'symbol': label, **summarize_graph(G, label)})

import networkx as nx
# Au-network composition (4 subgraphs connected by nuclei)
def build_au_network(n_subgraphs: int = 4, *, seed: int = 200):
    subs = [build_element_radial_graph(79, seed=seed+i) for i in range(n_subgraphs)]
    G = nx.Graph(); cur=-1; nuclei=[]
    for i, SG in enumerate(subs):
        off = cur+1; rel = nx.relabel_nodes(SG, {n: n+off for n in SG.nodes()}); G.update(rel); nuclei.append(off+0); cur = max(G.nodes())
    for i in range(len(nuclei)):
        a, b = nuclei[i], nuclei[(i+1)%len(nuclei)]
        G.add_edge(a, b)
    return G

try:
    Gnet = build_au_network()
    rows.append({'Z': 79, 'symbol': 'Au-net', **summarize_graph(Gnet, 'Au-network (n=4)')})
except Exception:
    pass

df = pd.DataFrame(rows)
df[['symbol','label','xi_c','mean_grad','mean_kphi','mean_path_length','u6_drift','u6_ok']].sort_values(['symbol'])

In [ ]:
# Save CSV/JSONL for parity
csv_path = os.path.join(OUTPUT_DIR, 'elements_signature_study.csv')
jsonl_path = os.path.join(OUTPUT_DIR, 'elements_signature_study.jsonl')
df[['symbol','label','xi_c','mean_grad','mean_kphi','mean_path_length','u6_drift','u6_ok']].to_csv(csv_path, index=False)
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")
csv_path, jsonl_path